In [42]:
import numpy as np
import random
import math
import pandas as pd
from sklearn.datasets import load_iris

In [49]:
iris = load_iris()
X = iris.data
y = iris.target

In [135]:
Kmax = 3
D = X.shape[1]
S = 100

In [29]:
class Bacteria:
   def __init__(self):
      self.position = None
      self.fitness = None
      self.health = None

In [47]:
X[random.randint(0, len(X))]

array([5.2, 3.5, 1.5, 0.2])

In [94]:
def encoding(Kmax, D):

    # Create a matrix to store the encoded positions.
    positions = np.zeros((Kmax, D))

    # Encode each position.
    for i in range(Kmax):
        # Generate a random position.
        position = np.random.uniform(0, 9, D)

        # Store the position and fitness value in the matrix.
        positions[i] = position

    return positions

In [98]:
X[0]

array([5.1, 3.5, 1.4, 0.2])

In [119]:
cluster1 = X[:50]
cluster2 = X[51:100]
cluster3 = X[101:150]

In [132]:
def objective_function(bacteria):
   con = sep = sum = 0
   for i in bacteria.position:
      for point in cluster1:
         dis1 = np.sum(np.linalg.norm(i - point)) / len(cluster1)
      for point in cluster2:
         dis2 = np.sum(np.linalg.norm(i - point)) / len(cluster2)
      for point in cluster3:
         dis3 = np.sum(np.linalg.norm(i - point)) / len(cluster3)
      con = dis1 + dis2 + dis3
   con += con

   sep = 1 / (np.linalg.norm(bacteria.position[0] - bacteria.position[1]) + np.linalg.norm(bacteria.position[0] - bacteria.position[2]) + np.linalg.norm(bacteria.position[1] - bacteria.position[2]))

   bacteria.fitness = con + sep

In [144]:
population = [Bacteria for _ in range(S)]
def init_population():
   for bacteria in population:
      positions = encoding(Kmax=Kmax, D=D)
      bacteria.position = positions
      objective_function(bacteria)

In [145]:
init_population()

In [147]:
population[0].position

array([[5.41672938, 3.23177343, 4.03438639, 0.37751628],
       [5.54526618, 3.26601009, 0.29341273, 5.0104517 ],
       [6.40556922, 6.26234023, 7.08529113, 8.31747748]])

In [146]:
population[0].fitness

1.1127010998444575

In [ ]:
def es_estimation(archive, nobj):

  minH = np.inf
  flag = 0
  count_state = 0

  # Calculate distances from ideal point
  # distances = get_distances_from_ideal(archive) 
  for bacteria in archive:
   #  distances = [(bacteria)] 

#   minA = min(distances)

  if flag == 0:

    if minH <= minA: # Situation 1
      count_state += 1
      
      if count_state >= nobj**2:
        flag = 1  

    else: # Situation 2
      minH = minA  
      count_state = 0

  else:
    
    if minH >= minA: # Situation 3
      count_state = 0
      flag = 0
      minH = minA

  return minH, flag, count_state

In [ ]:
min_val = [4.3, 2, 1, 0.1]
max_val = [7.9, 4.4, 6.9, 2.5]

In [ ]:
def select_gbests(archive, ideal_point):

  archive_size = len(archive)
  
  # Normalize objectives and calculate distances
  # normalize_archive(archive)  
  for bacterium in archive:
    distances = [find_Euclidean(bacterium, ideal_point)]

  # Select c_gbest
  c_gbest = archive[np.argmin(distances)]

  # Get crowding distances 
  # crowding_dists = crowding_distance(archive)

  # Sort by crowding distance  
  sorted_inds = np.argsort(crowding_dists)[::-1]  

  if archive_size >= 4:

    # Select first 4 non-dominated solutions
    # d_gbest_options = get_nondominated(archive, 4) 
    d_gbest_options = archive[:4]

    r = random.random()
    if r <= 0.25:
      d_gbest = d_gbest_options[0]
    elif r <= 0.5:  
      d_gbest = d_gbest_options[1]
    elif r <= 0.75:
      d_gbest = d_gbest_options[2]  
    else:  
      d_gbest = d_gbest_options[3]

  else:
    d_gbest = d_gbest_options[0]

  return c_gbest, d_gbest

In [ ]:
def select_pbest_leader(population):

  num_bacteria = len(population)
  pbest = [bact.position for bact in population] # get all pbest

  # Separately sort the 2 objectives of all pbest
  obj1_vals = [p[0] for p in pbest] 
  obj2_vals = [p[1] for p in pbest]

  sorted_obj1 = sorted(obj1_vals)
  sorted_inds1 = sorted(range(len(obj1_vals)), key=obj1_vals.__getitem__)

  sorted_obj2 = sorted(obj2_vals) 
  sorted_inds2 = sorted(range(len(obj2_vals)), key=obj2_vals.__getitem__)

  if random.random() < 0.5:
    sorted_pbest = sorted_obj1
    sorted_inds = sorted_inds1
  else:  
    sorted_pbest = sorted_obj2
    sorted_inds = sorted_inds2

  # Map sorted pbest to each bacterium
  learned_pbest = []
  for i in range(num_bacteria):
    pos = sorted_inds.index(i)  
    learned_pbest.append(sorted_pbest[pos])

  # Randomly select one as pbest leader  
  pbest_leader_idx = random.randint(0,num_bacteria-1)
  # pbest_leader = learned_pbest[pbest_leader_idx]
  pbest_leader = population[pbest_leader_idx]

  return pbest_leader

In [ ]:
def tumble(pbest_leader, gbest, bacterium, iter = 1):
    step_direction = c1 * random.random() * (pbest_leader.position[:-1] - bacterium.position[:-1]) + c2 * random.random() * (gbest.position[:-1] - bacterium.position[:-1]) * random.choice((-1, 1))
    
    C = c_min + math.exp(-cc * ((iter / N_s) ** 2)) * (c_max - c_min)

    bacterium.position[:-1] = bacterium.position[:-1] + C * step_direction

    # bacterium.step_direction = step_direction
    bacterium.step_size = C
    objective_function(bacterium)

In [ ]:
def swim(bacterium, pbest_leader, gbest):
    global best_value
    step_direction = c1 * random.random() * (pbest_leader.position[:-1]- bacterium.position[:-1]) + c2 * random.random() * (gbest.position[:-1] - bacterium.position[:-1])
    bacterium.position[:-1] = bacterium.position[:-1] + bacterium.step_size * step_direction
    for i in range(N_s):
        if bacterium.cost <= best_value:
            bacterium.position[:-1] = bacterium.position[:-1] + bacterium.step_size * step_direction
            objective_function(bacterium)
        else:
            break

In [ ]:
def chemotaxis(gbest, iter):
    global best_value, best_position, population

    for bacterium in population:
        pbest_leader = select_pbest_leader(population)
        tumble(pbest_leader,gbest, bacterium, iter)
        if bacterium.cost <= best_value:
            bacterium.pbest = bacterium.position
        last_cost = bacterium.cost
        swim(bacterium, pbest_leader, gbest)        
        if bacterium.cost <= last_cost:
            for i in range(N_s):
                swim(bacterium, pbest_leader, gbest)
                bacterium.pbest = bacterium.position
    
        objective_function(bacterium)

        if bacterium.cost <= best_value:
            best_value = bacterium.cost
            best_position = bacterium.position
        # if bacterium.cost > best_value:
        #     bacterium.cost = best_value
        #     bacterium.position = best_position

In [ ]:
def get_nondominated(population):
  
  nondominated = []
  
  # Compare each bacterium to others
  for i in range(len(population)):
    
    bacterium_i = population[i]
    is_nondominated = True
    
    # Compare bacterium i to all others    
    for j in range(len(population)):
      
      if i == j:
        continue
      
      bacterium_j = population[j]
      
      # If bacterium j has equal or better objective
      if bacterium_j.cost <= bacterium_i.cost:
        is_nondominated = False
        break
        
    # Bacterium i is nondominated against all others      
    if is_nondominated:
      nondominated.append(bacterium_i)
      
  return nondominated

In [ ]:
def get_dominated(population):

  dominated = []

  nondominated = get_nondominated(population)

  # Compare each bacterium to get nondominated
  for bacterium in population:

    is_dominated = True

    # Check if bacterium is in nondominated list
    if bacterium in nondominated:
      continue

    # Compare bacterium to all nondominated
    for nd_bacterium in nondominated:  

      if bacterium.cost > nd_bacterium.cost:
        is_dominated = True
        break

    # Bacterium is dominated by at least one nondom
    if is_dominated:
      dominated.append(bacterium)

  return dominated

In [ ]:
def reproduction_strategy(population, archive):
  
  # Get non-dominated solutions from archive and bacteria 
  nondom_archive = get_nondominated(archive) 
  nondom_bacteria = get_nondominated(population)
  
  Sum_nondom = len(nondom_archive) + len(nondom_bacteria)
  
  if Sum_nondom < len(population): # Situation 1
    
    BS1 = nondom_archive 
    BS2 = nondom_bacteria
    
    # Sort dominated bacteria and select some
    dominated = get_dominated(population)
    cost = [dominated[i].cost for i in range(len(dominated))] 
    sorted_indices = sorted(range(len(cost)), key=cost.__getitem__)
    new_dominated = [dominated[i] for i in sorted_indices]
    dominated = new_dominated.copy()
    
    BS3 = random.sample(dominated, len(population) - Sum_nondom)
    
    new_BS = BS1 + BS2 + BS3
    
  elif len(nondom_archive) < len(population): # Situation 2

    BS1 = nondom_archive
    BS2 = random.sample(nondom_bacteria, len(population) - len(nondom_archive))
    new_BS = BS1 + BS2
    
  else: # Situation 3

    BS1 = random.sample(nondom_archive, len(population))
    new_BS = BS1
    
  # Reproduce and return new swarm
  parents = new_BS[:len(new_BS)//2] 
  offspring = parents.copy()
  return new_BS + offspring

In [ ]:
def elimination_dispersal_BS(population, Ped):

  dim = len(population[0].position[:-1]) # dimension of each bacterium

  for i in range(len(population)):

    bacterium = population[i]

    if random.random() < Ped: 
      
      # Disperse bacterium location
      bacterium.position[:-1] = [min_val[j] + random.random() * (max_val[j]-min_val[j]) for j in range(dim)]
      
      # Limit to bounds and recalculate objective  
      bacterium.position[:-1] = [min(max(x,min_val[j]),max_val[j]) for j,x in enumerate(bacterium.position[:-1])]
      bacterium.cost = objective_function(bacterium)